
# Fine-tuning LoRA: Slides


## 1  Instalación de dependencias

In [1]:
# ────────────────────────────── 1. Dependencias de SO ─────────────────────────
!apt-get update -qq
!apt-get install -y poppler-utils libreoffice

# ────────────────────────────── 2. Paquetes Python ────────────────────────────
!pip install pdf2image pillow tqdm   # pdf2image necesita Poppler

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.9).
libreoffice is already the newest version (1:7.3.7-0ubuntu0.22.04.10).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [2]:
! pip install backoff transformers==4.49 peft
! pip install sacrebleu unbabel-comet

In [3]:
from google.colab import drive
import os

# Montar Google Drive
drive.mount('/content/drive')

# Rutas de los archivos zip en Google Drive
test_zip_path = '/content/drive/MyDrive/test.zip'
dev_zip_path = '/content/drive/MyDrive/dev.zip'
test_lst = '/content/drive/MyDrive/test.lst'
dev_lst = '/content/drive/MyDrive/dev.lst'

# Directorio de destino en Colab
colab_content_path = '/content/'

# Copiar los archivos zip a Colab
!cp "{test_zip_path}" "{colab_content_path}"
!cp "{dev_zip_path}" "{colab_content_path}"
!cp "{test_lst}" "{colab_content_path}"
!cp "{dev_lst}" "{colab_content_path}"

print(f"Archivos copiados a {colab_content_path}")
print(os.listdir(colab_content_path))

Mounted at /content/drive
Archivos copiados a /content/
['.config', 'drive', 'dev.lst', 'dev.zip', 'test.lst', 'test.zip', 'sample_data']


In [4]:
!unzip /content/test.zip -d /content/test
!unzip /content/dev.zip -d /content/dev

Archive:  /content/test.zip
   creating: /content/test/404/
   creating: /content/test/545006/
   creating: /content/test/596001/
   creating: /content/test/605000/
   creating: /content/test/606/
  inflating: /content/test/test.lst  
   creating: /content/test/606/manual_transcription/
   creating: /content/test/606/manual_translations/
  inflating: /content/test/606/slides.pptx  
  inflating: /content/test/606/606.m4a  
   creating: /content/test/606/manual_transcription/sentence_segmented/
  inflating: /content/test/606/manual_transcription/ie606.srt  
  inflating: /content/test/606/manual_transcription/ie606.txt  
  inflating: /content/test/606/manual_transcription/sentence_segmented/ie606.srt  
   creating: /content/test/606/manual_translations/de/
   creating: /content/test/606/manual_translations/es/
   creating: /content/test/606/manual_translations/fr/
   creating: /content/test/606/manual_translations/sl/
  inflating: /content/test/606/manual_translations/fr/606.lst  
  infla

## 2  Configuración principal

In [1]:
from transformers import AutoModelForCausalLM, AutoProcessor, GenerationConfig

import torch

model_name = 'microsoft/Phi-4-multimodal-instruct'
processor = AutoProcessor.from_pretrained(model_name,trust_remote_code = True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="cuda",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    # flash_attention_2 eager sdpa
    _attn_implementation='sdpa',
)

generation_config = GenerationConfig.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/auto/image_processing_auto.py:594: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow 

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

### 3.1  LoRA

In [2]:

from peft import LoraConfig, get_peft_model


lora_config = LoraConfig(
    r=8,                         # rango LoRA
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q_proj","k_proj"]
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 995,328 || all params: 5,575,455,552 || trainable%: 0.0179


/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(


### 3.2 Data

In [3]:
from pathlib import Path
import subprocess, soundfile as sf, datetime as dt

FFMPEG = "ffmpeg"          # o la ruta absoluta si no está en PATH

def m4a_to_wav(path_m4a, sr_out=16_000):
    """Convierte 1 × .m4a → .wav (mono, 16 kHz) y devuelve la ruta del WAV."""
    path_m4a = Path(path_m4a)
    wav_path = path_m4a.with_suffix(".wav")
    subprocess.run(
        [FFMPEG, "-loglevel", "error", "-y", "-i", str(path_m4a),
         "-ac", "1", "-ar", str(sr_out), str(wav_path)],
        check=True
    )
    return wav_path



def get_files(folder='test'):
  with open(f'/content/{folder}/{folder}.lst', 'r', encoding='utf-8') as f:
    files = f.read()
    files = files.splitlines()
    print(files)
    return files



In [4]:
from pathlib import Path
import tempfile, subprocess
from typing import List, Tuple
from PIL import Image
from pdf2image import convert_from_path

def _ppt_to_pdf(src: Path) -> Path:
    tmp = Path(tempfile.mkdtemp())
    pdf_out = tmp / f"{src.stem}.pdf"
    cmd = ["soffice", "--headless", "--convert-to", "pdf", "--outdir", str(tmp), str(src)]
    subprocess.run(cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return pdf_out  # se borrará cuando el tmp sea recolectado por el sistema

def _ensure_pdf(path: str | Path) -> Path:
    extensions = {".pdf", ".ppt", ".pptx"}
    ext = None
    for e in extensions:
        if Path(f"{path}{e}").exists():
            ext = e
            break
    p=Path(f"{path}{ext}")
    if ext == ".pdf":
        return p
    if ext in {".ppt", ".pptx"}:
        return _ppt_to_pdf(p)
    raise ValueError(f"Formato no soportado: {ext}. Usa .pdf/.ppt/.pptx")

# 2) Escribe PNGs al disco (menos RAM) y devuelve sus rutas
def _natural_sort_key(p: Path):
    # Ordena "page-2.png" < "page-10.png" de forma numérica
    stem = p.stem  # e.g., "page-12"
    try:
        n = int(stem.split("-")[-1])
    except Exception:
        n = 10**9  # si no hay número, mándalo al final
    return (stem.split("-")[0], n)

def slides_to_png_paths_resized(
    file_path: str | Path,
    out_dir: str | Path,
    size: Tuple[int, int] = (1600, 900),
    reuse_if_exists: bool = True  # ← controla si reutilizamos lo existente
) -> List[str]:
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    # Si ya hay PNGs y queremos reutilizar, devolverlos
    if reuse_if_exists:
        existing = sorted(out_dir.glob("*.png"), key=_natural_sort_key)
        if existing:
            return [str(p) for p in existing]
    pdf = _ensure_pdf(file_path)
    # Guardar directamente los PNG ya redimensionados
    paths = convert_from_path(pdf, size=size, fmt="png", output_folder=str(out_dir), paths_only=True)
    # pdf2image genera nombres tipo page-1.png; los ordenamos
    return [str(p) for p in sorted(map(Path, paths))]


In [5]:
from pathlib import Path
from typing import List, Dict, Tuple, Optional

def _ms_to_seconds(hms: str) -> float:
    m, s, _ = hms.split(":")
    return int(m)*60 + int(s)

def parse_slide_blocks(path: str | Path) -> Dict[str, Dict]:
    """
    Lee un archivo con múltiples bloques:
      <ID> <URL>
      <slide_id|None> <HH:MM:SS>
      ...
    Devuelve dict: { id: {"url": str, "timeline": [(t_sec, slide_id|None), ...]} }
    """
    p = Path(path)
    blocks: Dict[str, Dict] = {}
    cur_id = None
    for line in p.read_text().splitlines():
        line = line.strip()
        if not line:
            continue

        parts = line.split()
        # Cabecera: "<ID> <URL>"
        if len(parts) == 2 and parts[1].startswith("http"):
            cur_id = parts[0]
            blocks[cur_id] = []
            continue

        # Línea de cambio: "<slide_id|None> <HH:MM:SS>"
        if cur_id is not None and len(parts) == 2 and ":" in parts[1]:
            slide_raw, hms = parts
            slide_id = None if slide_raw == "None" else int(slide_raw)
            t = _ms_to_seconds(hms)
            blocks[cur_id].append((t, slide_id))

    # Ordena por tiempo y asegura que hay un evento en t=0
    for k,v in blocks.items():
        v.sort(key=lambda x: x[0])

    return blocks

def build_slide_intervals(timeline: List[Tuple[float, Optional[int]]]) -> List[Dict]:
    """
    Convierte una timeline en intervalos contiguos: [{"start":t0,"end":t1,"slide":id}, ...]
    El último intervalo termina en +inf (None).
    """
    out = []
    for i, (t, slide) in enumerate(timeline):
        t_next = timeline[i+1][0] if i+1 < len(timeline) else None
        out.append({"start": t, "end": t_next, "slide": slide})
    return out


In [6]:
def slide_at(time_s: float, intervals: List[Dict]) -> Optional[int]:
    """Devuelve la slide activa en 'time_s'."""
    # búsqueda lineal (puedes optimizar con bisect si quieres)
    for seg in intervals:
        if seg["end"] is None:  # último intervalo
            return seg["slide"]
        if seg["start"] <= time_s < seg["end"]:
            return seg["slide"]
    return None

from typing import List, Optional

def slides_in_interval(
    start: float,
    end: float,
    intervals: List[dict],
    *,
    min_fraction: float = 0.10,   # % mínimo del segmento que debe cubrir la slide
    min_seconds: float = 0.0,     # y/o segundos absolutos mínimos de cobertura
    slack: float = 0.25,          # margen (±s) para tolerar pequeñas imprecisiones
    sort_by_coverage: bool = True,
    top_k: Optional[int] = 1
) -> List[int]:
    """
    Devuelve las slides que realmente 'cuentan' en [start, end), filtrando
    las que cubren menos de min_fraction del segmento o menos de min_seconds.

    - 'slack' expande el segmento a [start-slack, end+slack] para evitar cortar
      por milésimas (timings manuales, FPS, etc.).
    - Si sort_by_coverage=True, ordena por mayor cobertura; si no, por aparición.
    - 'top_k' limita el nº de slides devueltas (tras el filtro).
    """
    seg_len = max(1e-6, end - start)  # evita división por cero
    seg_lo = start - slack
    seg_hi = end + slack

    cover_by_slide = {}   # slide_id -> (cover_seconds, first_order)

    order = 0
    for seg in intervals:
        slide = seg["slide"]
        if slide is None:
            continue

        s_lo = seg["start"]
        s_hi = float("inf") if seg["end"] is None else seg["end"]

        # intersección con margen
        lo = max(seg_lo, s_lo)
        hi = min(seg_hi, s_hi)
        overlap = max(0.0, hi - lo)
        if overlap <= 0:
            continue

        if slide not in cover_by_slide:
            cover_by_slide[slide] = [0.0, order]
            order += 1
        cover_by_slide[slide][0] += overlap

    # Filtro por cobertura
    out = []
    for slide, (cover_s, first_order) in cover_by_slide.items():
        frac = cover_s / seg_len
        if cover_s >= min_seconds or frac >= min_fraction:
            out.append((slide, cover_s, first_order))

    if not out:
        return []

    # Orden
    if sort_by_coverage:
        out.sort(key=lambda x: (-x[1], x[2]))   # más cobertura primero
    else:
        out.sort(key=lambda x: x[2])            # por aparición

    if top_k is not None:
        out = out[:top_k]

    return [slide for slide, _, _ in out]


In [7]:
from pathlib import Path
import datasets as ds


def build_dataset(folder="train", language="es", files=None,
                  dpi=150, target_size=(1600,900), mode="letterbox",
                  bg="white", store_slide_paths=False):
    rows = []
    files = get_files(folder) if files is None else files

    # Carga y parsea el maestro (una vez)
    blocks = parse_slide_blocks(f'{folder}.lst')

    for name in files:
        base = Path(f"/content/{folder}/{name}/manual_translations/{language}")
        with open(base / f"{name}.lst") as f:
            times = [(float(a), float(b)) for a,b in (ln.split() for ln in f.read().splitlines())]
        with open(base / f"{name}.en") as f:
            src = f.read().splitlines()
        with open(base / f"{name}.{language}") as f:
            tgt = f.read().splitlines()
        assert len(times) == len(src) == len(tgt)

        # 1) Preprocesa slides una vez (o usa caché si ya existen)
        slides_root = Path(f"/content/{folder}/{name}/slides")
        paths=slides_to_png_paths_resized(slides_root,f'/content/{folder}/{name}/slides/')
        slide_paths = [str(Path(p).relative_to(slides_root)) for p in paths]
        print(paths)
        print(slide_paths)

        # 2) Audio (como ya hacías)
        wav_path = str(m4a_to_wav(f"/content/{folder}/{name}/{name}.m4a"))

        if str(name) in blocks:
            intervals = build_slide_intervals(blocks[str(name)])
        else:
            intervals = [{"start":0.0,"end":None,"slide":None}]
            print(f"[WARN] No hay bloque de slides para {name}")

        # 3) Crea filas por segmento reusando las mismas referencias a slides
        for (s, e), txt_in, txt_out in zip(times, src, tgt):
            slides  = slides_in_interval(s, e, intervals)
            row = {
                "wav_path": wav_path,
                "start": s,
                "end": e,
                "source_text": txt_in,
                "target_text": txt_out,
                "slide_paths": paths[slides[0]] if slides else None,

            }
            rows.append(row)

    return ds.Dataset.from_list(rows)

train_ds = build_dataset("dev", files=['500011', '624000', '609'])
val_ds   = build_dataset("dev", files=['550000','592'])
test_ds   = build_dataset("test")

['/content/dev/500011/slides/ab838ea2-ea51-45dd-b036-f488464840b1-01.png', '/content/dev/500011/slides/ab838ea2-ea51-45dd-b036-f488464840b1-02.png', '/content/dev/500011/slides/ab838ea2-ea51-45dd-b036-f488464840b1-03.png', '/content/dev/500011/slides/ab838ea2-ea51-45dd-b036-f488464840b1-04.png', '/content/dev/500011/slides/ab838ea2-ea51-45dd-b036-f488464840b1-05.png', '/content/dev/500011/slides/ab838ea2-ea51-45dd-b036-f488464840b1-06.png', '/content/dev/500011/slides/ab838ea2-ea51-45dd-b036-f488464840b1-07.png', '/content/dev/500011/slides/ab838ea2-ea51-45dd-b036-f488464840b1-08.png', '/content/dev/500011/slides/ab838ea2-ea51-45dd-b036-f488464840b1-09.png', '/content/dev/500011/slides/ab838ea2-ea51-45dd-b036-f488464840b1-10.png', '/content/dev/500011/slides/ab838ea2-ea51-45dd-b036-f488464840b1-11.png', '/content/dev/500011/slides/ab838ea2-ea51-45dd-b036-f488464840b1-12.png', '/content/dev/500011/slides/ab838ea2-ea51-45dd-b036-f488464840b1-13.png', '/content/dev/500011/slides/ab838ea2-

In [8]:
print(train_ds)
print(val_ds)
print(test_ds)

Dataset({
    features: ['wav_path', 'start', 'end', 'source_text', 'target_text', 'slide_paths'],
    num_rows: 708
})
Dataset({
    features: ['wav_path', 'start', 'end', 'source_text', 'target_text', 'slide_paths'],
    num_rows: 742
})
Dataset({
    features: ['wav_path', 'start', 'end', 'source_text', 'target_text', 'slide_paths'],
    num_rows: 1405
})


In [9]:
PROMPT_TEXT="<|audio_1|>\n<|image_1|>\nTranslate the audio to es-ES."

In [10]:
import torch, soundfile as sf
from PIL import Image

class CollatorPhi4Audio:
    def __init__(self, processor, answer_suffix="<|end|><|endoftext|>",
                 sr=16_000, ignore_idx=-100, cache=True,
                 force_image=True, dummy_size=(1600, 900), dummy_color="white"):
        self.proc = processor
        self.sr = sr
        self.pad_id = processor.tokenizer.pad_token_id
        self.suffix = answer_suffix
        self.ignore = ignore_idx
        self.cache_fd = {} if cache else None
        # imagen dummy para filas sin slide
        self.force_image = force_image
        self.dummy_image = Image.new("RGB", dummy_size, dummy_color)


    # No reabrir el WAV completo en cada segmento
    def _read_segment(self, path, start, end):
        if self.cache_fd is not None:
            if path not in self.cache_fd:
                self.cache_fd[path] = sf.SoundFile(path)
            f = self.cache_fd[path]
        else:
            f = sf.SoundFile(path)
        frames = int((end - start) * self.sr)
        f.seek(int(start * self.sr))
        audio = f.read(frames, dtype="float32")
        return audio

    def _pick_image(self, ex):
        """
        Devuelve una PIL.Image si hay slide; en caso contrario:
        - si force_image=True → imagen dummy
        - si force_image=False → None (¡no usar si mezclas en el batch!)
        """
        sp = ex.get("slide_paths")  # puede ser str, list o None
        if isinstance(sp, list):
            sp = sp[0] if sp else None
        if isinstance(sp, str):
            return Image.open(sp).convert("RGB")
        return self.dummy_image if self.force_image else None

    def __call__(self, batch):
        prompts, answers, wavs, images = [], [], [], []

        # Construimos SIEMPRE prompt con <|image_1|> si usamos dummy
        use_img_token = self.force_image or any(bool(b.get("slide_paths")) for b in batch)
        for ex in batch:


            prompt = self.proc.tokenizer.apply_chat_template(
                [{"role": "user", "content": PROMPT_TEXT}],
                tokenize=False, add_generation_prompt=True
            )
            prompts.append(prompt)
            answers.append(ex["target_text"] + self.suffix)

            wavs.append((self._read_segment(ex["wav_path"], ex["start"], ex["end"]), self.sr))
            img = self._pick_image(ex)
            if not use_img_token and img is not None:
                # si decidiste no usar <|image_1|>, no pases imagen (caso poco común)
                img = None
            images.append(img)

        # ---- Texto (prompt + respuesta) ---------------------------------
        tok_in  = self.proc.tokenizer(prompts,  return_tensors="pt", padding=True)
        tok_ans = self.proc.tokenizer(answers, return_tensors="pt", padding=True)

        input_ids = torch.cat([tok_in.input_ids, tok_ans.input_ids], dim=1)
        attn_mask = (input_ids != self.pad_id).long()

        labels = torch.full_like(input_ids, self.ignore)
        ans_len = tok_ans.input_ids.size(1)
        labels[:, -ans_len:] = tok_ans.input_ids

        # (Opcional) enmascarar CoT si lo usas en answers y supervise_cot=False
        # ... (puedes pegar aquí el bloque de máscara de CoT adaptado a tu formato)

        # ---- Embeddings multimodales (UNA sola llamada) -----------------
        kwargs = dict(text=prompts, audios=wavs, return_tensors="pt", padding=True)
        if use_img_token:
            # Asegúrate de que no haya None en images si pones <|image_1|>
            images = [im if im is not None else self.dummy_image for im in images]
            kwargs["images"] = images

        feats = self.proc(**kwargs)  # genera input_audio_embeds / input_image_embeds

        batch_out = {
            "input_ids":            input_ids,
            "labels":               labels,
            "attention_mask":       attn_mask,
            "input_audio_embeds":   feats["input_audio_embeds"],
            "audio_attention_mask": feats.get("audio_attention_mask"),
            "input_mode":           torch.ones(len(batch), dtype=torch.long) * 3 ,
        }
        if use_img_token:
            batch_out.update({
                "input_image_embeds":   feats["input_image_embeds"],
                "image_attention_mask": feats.get("image_attention_mask"),
            })
        return batch_out


In [12]:
train_dataloader = torch.utils.data.DataLoader(
    train_ds,
    shuffle=True,
    batch_size=4,
    collate_fn=CollatorPhi4Audio(processor),
    pin_memory=True,
)
val_dataloader = torch.utils.data.DataLoader(
    val_ds,
    shuffle=False,
    batch_size=4,
    collate_fn=CollatorPhi4Audio(processor),
    pin_memory=True,
)

## 3  Entrenamiento

### 3.1 Helpers

In [13]:
import math, os, gc, torch
from tqdm import tqdm
from typing import Dict, List, Tuple

def move_batch(batch, device):
    return {
        k: v.to(device, dtype=torch.bfloat16 if v.dtype == torch.float else torch.long)
        for k, v in batch.items()
    }

@torch.no_grad()
def avg_loss_on_loader(model, dataloader, device):
    model.eval()
    losses, n = 0.0, 0
    for batch in dataloader:
        batch = move_batch(batch, device)
        with torch.autocast(device_type="cuda", dtype=torch.bfloat16):
            out = model(**batch)
        losses += out.loss.item() * batch[list(batch.keys())[0]].size(0)
        n += batch[list(batch.keys())[0]].size(0)
    model.train()
    return losses / n


### 3.2 Eval Definition

In [14]:
import soundfile as sf
from collections import defaultdict

_open_wavs = {}                  # caché: path → SoundFile

def read_segment(path, start, end, sr=16_000):
    """Lee [start,end] s de un WAV grande sin cargarlo entero."""
    if path not in _open_wavs:
        _open_wavs[path] = sf.SoundFile(path)
    f = _open_wavs[path]
    frames = int((end - start) * sr)
    f.seek(int(start * sr))
    return f.read(frames, dtype="float32")
def segments_by_audio(ds):
    groups = defaultdict(list)
    for ex in ds:
        groups[ex["wav_path"]].append(ex)
    # ordenamos cada audio por inicio temporal
    for segs in groups.values():
        segs.sort(key=lambda x: x["start"])
    return groups


In [15]:
val_groups  = segments_by_audio(val_ds)

In [16]:
from collections import defaultdict
from itertools import chain
import torch, gc
from tqdm import tqdm

device        = torch.device("cuda:0")
batch_size    = 2

user_prompt      = '<|user|>'
assistant_prompt = '<|assistant|>'
prompt_suffix    = '<|end|>'

CHAT_PROMPT = f'{user_prompt}{PROMPT_TEXT}{prompt_suffix}{assistant_prompt}'



def generate_output(groups):
    hyps_all, refs_all, srcs_all = [], [], []

    for wav_path, segs in tqdm(groups.items(), desc="Audios"):
        hyps, refs, srcs = [], [], []

        # dividimos en lotes (batch_size) para GPU
        for i in range(0, len(segs), batch_size):
            chunk = segs[i:i+batch_size]

            wavs   = [(read_segment(s["wav_path"], s["start"], s["end"]),16_000) for s in chunk]
            images = [Image.open(s["slide_paths"]).convert("RGB") if s.get("slide_paths") else Image.new("RGB", (1600, 900), "white") for s in chunk]
            prompts= [CHAT_PROMPT] * len(chunk)

            inputs = processor(text=prompts, audios=wavs, images=images,
                               return_tensors="pt", padding=True
                               ).to(device)

            with torch.inference_mode(), torch.autocast("cuda"):
                gen_ids = model.generate(**inputs, generation_config=generation_config, max_new_tokens=1000, num_logits_to_keep=0)

            # quitamos el prompt textual
            gen_ids = gen_ids[:, inputs["input_ids"].shape[1]:]

            hyps.extend(
                processor.batch_decode(gen_ids.cpu(),
                                       skip_special_tokens=True,
                                       clean_up_tokenization_spaces=False)
            )

            # limpieza VRAM
            # for im in images: im.close() # This line might cause an error if Image.new is used.
            del inputs, gen_ids # , auds, imgs, prompts # auds and imgs are not defined here
            torch.cuda.empty_cache(); gc.collect()

            refs.extend([s["target_text"] for s in chunk])
            srcs.extend([s["source_text"] for s in chunk])

        print(hyps)
        print(refs)
        print(srcs)


        hyps_all.append(hyps)
        refs_all.append(refs)
        srcs_all.append(srcs)

    return hyps_all, refs_all, srcs_all

In [17]:
from __future__ import annotations

from itertools import chain
from typing import Callable, Dict, List, Tuple

import sacrebleu
from comet import download_model, load_from_checkpoint

# ────────────────────────────────────────────────────────────────────────────────
#  Cargamos (y cacheamos) el modelo COMET‑22 una sola vez
# ────────────────────────────────────────────────────────────────────────────────
_COMET_MODEL = None
_COMET_PATH = None
_MODEL_NAME = "Unbabel/wmt22-comet-da"


def _get_comet_model(gpus: int = 0):
    """Devuelve un modelo COMET‑22 listo para `predict` (cacheado)."""
    global _COMET_MODEL, _COMET_PATH

    if _COMET_MODEL is None:
        _COMET_PATH = download_model(_MODEL_NAME)  # se guarda en ~/.cache
        _COMET_MODEL = load_from_checkpoint(_COMET_PATH)

    # *Nota*: el parámetro `gpus` se pasa a `.predict` y **no** aquí, pero
    # exponemos el arg para quien quiera forzar CPU en la firma pública.
    return _COMET_MODEL


# ────────────────────────────────────────────────────────────────────────────────
#  Función principal
# ────────────────────────────────────────────────────────────────────────────────

def bleu_comet_by_audio(
    refs_audio: List[List[str]],
    hyps_audio: List[List[str]],
    srcs_audio: List[List[str]],
    transform: Callable[[str], str] = lambda x: x,
    comet_gpus: int = 0,
    comet_batch_size: int = 8,
) -> Tuple[Dict[str, float], List[Dict[str, float]]]:
    """Calcula BLEU y COMET‑22, **global** y **por audio**.

    Parameters
    ----------
    refs_audio, hyps_audio, srcs_audio : list[list[str]]
        Listas anidadas con el mismo nº de audios y segmentos.
    transform : callable
        Función de normalización por frase (identidad por defecto).
    comet_gpus : int
        Nº de GPUs a usar en `model.predict` (0 ⇒ CPU).
    comet_batch_size : int
        Tamaño de lote para COMET (trade‑off velocidad / memoria).

    Returns
    -------
    (global_metrics, per_audio_metrics)
        global_metrics    = {"bleu": float, "comet22": float}
        per_audio_metrics = [
            {"audio_id": i, "bleu": float, "comet22": float},
            ...
        ]
    """

    # ── Comprobaciones básicas ──────────────────────────────────────────────
    assert len(refs_audio) == len(hyps_audio) == len(srcs_audio), (
        "refs, hyps y srcs deben tener la misma longitud (nº audios)"
    )

    per_audio: List[Dict[str, float]] = []

    # ── Recorremos audio por audio ──────────────────────────────────────────
    comet_model = _get_comet_model(gpus=comet_gpus)

    for idx, (ref_seg, hyp_seg, src_seg) in enumerate(
        zip(refs_audio, hyps_audio, srcs_audio)
    ):
        assert len(ref_seg) == len(hyp_seg) == len(src_seg), (
            f"El audio {idx} contiene diferente nº de segmentos"
        )

        ref_seg = [transform(r) for r in ref_seg]
        hyp_seg = [transform(h) for h in hyp_seg]
        src_seg = [transform(s) for s in src_seg]

        # ── BLEU corpus‑level para el audio ────────────────────────────────
        bleu_score = sacrebleu.corpus_bleu(hyp_seg, [ref_seg]).score

        # ── COMET‑22 ───────────────────────────────────────────────────────
        samples = [  # una entrada por segmento
            {"src": s, "mt": h, "ref": r}
            for s, h, r in zip(src_seg, hyp_seg, ref_seg)
        ]
        comet_out = comet_model.predict(
            samples,
            batch_size=comet_batch_size,
            gpus=comet_gpus,
            progress_bar=False,
        )
        comet_score = comet_out["system_score"]  # media ya calculada

        per_audio.append(
            {
                "audio_id": idx,
                "bleu": bleu_score,
                "comet22": comet_score,
            }
        )

    # ── Métricas globales ──────────────────────────────────────────────────
    refs_all = list(chain.from_iterable(refs_audio))
    hyps_all = list(chain.from_iterable(hyps_audio))
    srcs_all = list(chain.from_iterable(srcs_audio))

    bleu_global = sacrebleu.corpus_bleu(hyps_all, [refs_all]).score

    comet_samples = [
        {"src": s, "mt": h, "ref": r}
        for s, h, r in zip(srcs_all, hyps_all, refs_all)
    ]
    comet_global = comet_model.predict(
        comet_samples,
        batch_size=comet_batch_size,
        gpus=comet_gpus,
        progress_bar=False,
    )["system_score"]

    return {"bleu": bleu_global, "comet22": comet_global}, per_audio





In [18]:
def eval_bleu_comet_fn_val(groups_val, processor, model, generation_config, device,
                           batch_size=8, transform=lambda x: x):
    # 1) Generar hipótesis
    hyps, refs, srcs = generate_output(groups_val)  # usa tu función tal cual

    # 2) Calcular métricas
    global_metrics, per_audio = bleu_comet_by_audio(
        refs_audio=refs,
        hyps_audio=hyps,
        srcs_audio=srcs,
        transform=transform,
        comet_gpus=1,        # o 0 si prefieres CPU
        comet_batch_size=8
    )
    return global_metrics, per_audio


### 3.3 Train Val

In [36]:
import os, math, gc, torch
from tqdm import tqdm

def disable_vision_checkpointing_phi4mm(model):
    # Apaga cualquier GC global
    if hasattr(model, "gradient_checkpointing_disable"):
        model.gradient_checkpointing_disable()
    # Fuerza OFF en módulos de visión (SigLIP/NaViT)
    for m in model.modules():
        cls = m.__class__.__name__.lower()
        if any(k in cls for k in ("siglip", "navit", "vision")):
            if hasattr(m, "gradient_checkpointing"):
                m.gradient_checkpointing = False
            if hasattr(m, "use_gradient_checkpointing"):
                m.use_gradient_checkpointing = False

def train(
    model,
    train_loader,
    val_loader,
    optimizer,
    device,
    acc_steps=2,
    max_epochs=5,
    clip_grad=1.0,
    scheduler=None,
    scheduler_type=None,
    ckpt_dir="checkpoints",
    patience=3,
    eval_bleu_comet_fn=None,
    eval_bleu_args=None,
    verbose_every=20
):
    os.makedirs(ckpt_dir, exist_ok=True)
    best_metric = -math.inf
    no_improve = 0

    model.to(device)
    # 1) Apaga KV cache para train y GC de visión (clave para evitar el AttributeError)
    if hasattr(model, "config") and hasattr(model.config, "use_cache"):
        model.config.use_cache = False
    disable_vision_checkpointing_phi4mm(model)

    model.train()
    optimizer.zero_grad(set_to_none=True)

    for epoch in range(1, max_epochs + 1):
        print(f"\nEpoch {epoch}/{max_epochs}")
        running = 0.0
        model.train()

        for step, batch in enumerate(tqdm(train_loader)):
            batch = move_batch(batch, device)

            with torch.autocast("cuda", dtype=torch.bfloat16):
                outputs = model(**batch)
                loss = outputs.loss / acc_steps

            loss.backward()
            running += loss.item() * acc_steps

            if (step + 1) % acc_steps == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), clip_grad)
                optimizer.step()
                optimizer.zero_grad(set_to_none=True)

                if scheduler is not None and scheduler_type != "plateau":
                    # Cosine, etc., si quieres por step (opcional)
                    pass

            if step % verbose_every == 0:
                print(f" step {step:<4} | loss = {loss.item() * acc_steps:.4f}")

        # ---------- VALIDACIÓN ----------
        model.eval()
        # (GC de visión ya está OFF; no lo re-actives)
        val_loss = avg_loss_on_loader(model, val_loader, device)
        print(f"  -> val_loss: {val_loss:.4f}")

        # Métricas opcionales (BLEU/COMET)
        extra_metrics = {}
        if eval_bleu_comet_fn is not None:
            # Puedes activar cache solo para generación dentro de eval_fn si quieres
            old_cache = getattr(model.config, "use_cache", False)
            if hasattr(model.config, "use_cache"):
                model.config.use_cache = True
            metrics_global, _ = eval_bleu_comet_fn(**(eval_bleu_args or {}))
            if hasattr(model.config, "use_cache"):
                model.config.use_cache = old_cache

            print(f"  -> BLEU: {metrics_global['bleu']:.2f} | COMET22: {metrics_global['comet22']:.3f}")
            extra_metrics = metrics_global
            current_key_metric = metrics_global["comet22"]
        else:
            current_key_metric = -val_loss

        # ---------- Scheduler step ----------
        if scheduler is not None:
            if scheduler_type == "plateau":
                scheduler.step(val_loss)
            else:
                scheduler.step()

        # ---------- Checkpoint & Early stopping ----------
        if current_key_metric > best_metric:
            best_metric = current_key_metric
            no_improve = 0
            path = os.path.join(ckpt_dir, f"best_epoch{epoch}.pt")
            torch.save({
                "model_state": model.state_dict(),
                "optimizer_state": optimizer.state_dict(),
                "epoch": epoch,
                "val_loss": val_loss,
                **extra_metrics
            }, path)
            print(f"  ✔ Nuevo mejor modelo guardado en {path}")
        else:
            no_improve += 1
            print(f"  (sin mejora {no_improve}/{patience})")
            if no_improve >= patience:
                print("  ✖ Early stopping activado.")
                break

        torch.cuda.empty_cache(); gc.collect()


In [37]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
eval_args = dict(
    groups_val=val_groups,                # dict {wav_path: [segs...]}
    processor=processor,
    model=model,
    generation_config=generation_config,
    device=device,
    batch_size=8,
    transform=lambda x: x
)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)


scheduler = ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=1, min_lr=1e-7)

train(model, train_dataloader, val_dataloader, optimizer, device,
      scheduler=scheduler, scheduler_type="plateau",
      acc_steps=8, max_epochs=10, patience=3,)
      # eval_bleu_comet_fn=eval_bleu_comet_fn_val,
      # eval_bleu_args=eval_args)



Epoch 1/10


  0%|          | 0/177 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
  1%|          | 1/177 [00:01<04:21,  1.49s/it]

 step 0    | loss = 4.3333


 12%|█▏        | 21/177 [00:18<02:12,  1.17it/s]

 step 20   | loss = 2.4639


...

 step 140  | loss = 1.3635


 91%|█████████ | 161/177 [02:11<00:13,  1.18it/s]

 step 160  | loss = 1.2751


100%|██████████| 177/177 [02:24<00:00,  1.22it/s]


  -> val_loss: 2.0274
  (sin mejora 3/3)
  ✖ Early stopping activado.


## 4  Evaluación  (BLEU y COMET)

### Evaluacion completa

In [19]:

torch.cuda.empty_cache(); gc.collect()


# Cargar checkpoint
ckpt = torch.load("checkpoints/best_epoch2.pt", map_location=device)
model.load_state_dict(ckpt["model_state"], strict=False)


model.gradient_checkpointing_disable()          # importante para evitar el bug/anular GC
model.eval()
model.config.use_cache = True                   # más memoria, pero más rápido


In [20]:
# Carga mejor checkpoint si quieres

groups  = segments_by_audio(test_ds)
print(f"{len(groups)=} audios")
hyps, refs, srcs = generate_output(groups)



len(groups)=5 audios


Audios:   0%|          | 0/5 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Audios:  20%|██        | 1/5 [20:45<1:23:02, 1245.63s/it]

['Así que, si tienes alguna pregunta, no dudes en ponerte en contacto con nosotros.', 'Este es el resumen de la sesión 404.', 'Así que, si tienes una paciente que tiene una enfermedad muy avanzada, y tienes una opción de cirugía, y tienes una opción de quimioterapia, y tienes una opción de radioterapia, y tienes una opción de quimioterapia, y tienes una opción de radioterapia, y tienes una opción de quimioterapia, y tienes una opción de radioterapia, y tienes una opción de quimioterapia, y tienes una opción de radioterapia, y tienes una opción de quimioterapia, y tienes una opción de radioterapia, y tienes una opción de quimioterapia, y tienes una opción de radioterapia, y tienes una opción de quimioterapia, y tienes una opción de radioterapia, y tienes una opción de quimioterapia, y tienes una opción de radioterapia, y tienes una opción de quimioterapia, y tienes una opción de radioterapia, y tienes una opción de quimioterapia, y tienes una opción de radioterapia, y tienes una opción 

Audios:  40%|████      | 2/5 [1:12:40<1:57:15, 2345.29s/it]

['Así que, si tenemos una paciente que tiene una enfermedad muy localizada, y tenemos una enfermedad muy localizada, y tenemos una enfermedad muy localizada, y tenemos una enfermedad muy localizada, y tenemos una enfermedad muy localizada, y tenemos una enfermedad muy localizada, y tenemos una enfermedad muy localizada, y tenemos una enfermedad muy localizada, y tenemos una enfermedad muy localizada, y tenemos una enfermedad muy localizada, y tenemos una enfermedad muy localizada, y tenemos una enfermedad muy localizada, y tenemos una enfermedad muy localizada, y tenemos una enfermedad muy localizada, y tenemos una enfermedad muy localizada, y tenemos una enfermedad muy localizada, y tenemos una enfermedad muy localizada, y tenemos una enfermedad muy localizada, y tenemos una enfermedad muy localizada, y tenemos una enfermedad muy localizada, y tenemos una enfermedad muy localizada, y tenemos una enfermedad muy localizada, y tenemos una enfermedad muy localizada, y tenemos una enfermed

...

['Y si quieres compartir tu experiencia e-Eso, puedes usar el hashtag #e_ES.', 'Y, en resumen, tenemos que hacer más investigación en este campo.', 'Y, espero, que tendremos algunas preguntas interesantes que surjan como también.', 'Así que, si tenemos una paciente que tiene una enfermedad muy avanzada, y tenemos una paciente que tiene una enfermedad muy avanzada, y tenemos una paciente que tiene una enfermedad muy avanzada, y tenemos una paciente que tiene una enfermedad muy avanzada, y tenemos una paciente que tiene una enfermedad muy avanzada, y tenemos una paciente que tiene una enfermedad muy avanzada, y tenemos una paciente que tiene una enfermedad muy avanzada, y tenemos una paciente que tiene una enfermedad muy avanzada, y tenemos una paciente que tiene una enfermedad muy avanzada, y tenemos una paciente que tiene una enfermedad muy avanzada, y tenemos una paciente que tiene una enfermedad muy avanzada, y tenemos una paciente que tiene una enfermedad muy avanzada, y tenemos una

Audios:  80%|████████  | 4/5 [2:29:43<38:09, 2289.60s/it]  

['Así que, si quieres compartir tu experiencia de e-ESO, puedes usar el hashtag #e_ES.', 'Y si tienes alguna pregunta, no dudes en ponerte en contacto con nosotros.', 'Así que, en resumen, tenemos aquí expertos de todo el mundo que se reunirán para discutir la multidisciplinariedad, y cómo podemos mejorar el abordaje del cáncer rectal para mejorar el resultado.', 'Y tenemos dos expertos que nos darán su presentación: un oncólogo médico, Dra. Katia Roque Perez, y un oncólogo de radioterapia, Profesora Maria Antonietta Gambacorta.', 'Así que comenzamos con la Dra. Katia Roque Perez que dará una visión de la oncología médica en el cáncer rectal y después, terminaremos con la Dra. Gamba Corta que dará una visión de la oncología de radioterapia en el cáncer rectal.', 'Y en el final, estamos abiertos a preguntas y a discutir qué piensan que es pertinente para esta sección.', 'Katia, por favor, vaya adelante.', 'Así que, en resumen, tenemos que tener una estrategia de gestión multidisciplinar

Audios: 100%|██████████| 5/5 [2:47:03<00:00, 2004.69s/it]

['Así que, estoy hablando sobre la importancia de la atención multidisciplinaria.', 'Así que, como escuchaste, mi nombre es Ann Partridge.', 'Y si tienes una paciente que tiene una enfermedad muy avanzada, y tienes una paciente que tiene una enfermedad muy avanzada, y tienes una paciente que tiene una enfermedad muy avanzada, y tienes una paciente que tiene una enfermedad muy avanzada, y tienes una paciente que tiene una enfermedad muy avanzada, y tienes una paciente que tiene una enfermedad muy avanzada, y tienes una paciente que tiene una enfermedad muy avanzada, y tienes una paciente que tiene una enfermedad muy avanzada, y tienes una paciente que tiene una enfermedad muy avanzada, y tienes una paciente que tiene una enfermedad muy avanzada, y tienes una paciente que tiene una enfermedad muy avanzada, y tienes una paciente que tiene una enfermedad muy avanzada, y tienes una paciente que tiene una enfermedad muy avanzada, y tienes una paciente que tiene una enfermedad muy avanzada, y

In [21]:
global_metrics, per_audio = bleu_comet_by_audio(refs, hyps, srcs,
                                                comet_gpus=1, comet_batch_size=8)





Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

checkpoints/model.ckpt:   0%|          | 0.00/2.32G [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

hparams.yaml:   0%|          | 0.00/567 [00:00<?, ?B/s]

LICENSE: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/sta

In [22]:

print("BLEU  global :", global_metrics["bleu"])
print("COMET-22    :", global_metrics["comet22"])

BLEU  global : 9.848251250799668
COMET-22    : 0.7155068239813598


In [23]:
per_audio

[{'audio_id': 0, 'bleu': 15.07580035296114, 'comet22': 0.6857664934330923},
 {'audio_id': 1, 'bleu': 7.488198109810063, 'comet22': 0.7247044376038403},
 {'audio_id': 2, 'bleu': 10.703539358215002, 'comet22': 0.7257168796918204},
 {'audio_id': 3, 'bleu': 9.4050675199903, 'comet22': 0.7089780619863267},
 {'audio_id': 4, 'bleu': 8.926125719706834, 'comet22': 0.7052158912022909}]